In [0]:
import numpy as np
import torch
from torch import nn
from torch.autograd import Variable
from torchvision.datasets import CIFAR10

In [0]:
def vgg_block(num_convs, in_channels, out_channels):
    net = [nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),nn.ReLU(True)]
    
    for i in range(num_convs - 1):
        net.append(nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1))
        net.append(nn.ReLU(True))
    
    net.append(nn.MaxPool2d(2,2))
    return nn.Sequential(*net)

In [3]:
block_demo = vgg_block(3, 64, 128)
print(block_demo)

Sequential(
  (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU(inplace)
  (2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): ReLU(inplace)
  (4): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (5): ReLU(inplace)
  (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
)


In [4]:
input_demo = Variable(torch.zeros(1,64,300,300))
output_demo = block_demo(input_demo)
print(output_demo.shape)

torch.Size([1, 128, 150, 150])


In [0]:
def vgg_stack(num_convs, channels):
    net = []
    for n,c in zip(num_convs, channels):
        in_c = c[0]
        out_c =c[1]
        net.append(vgg_block(n, in_c, out_c))
    return nn.Sequential(*net)

In [6]:
vgg_net = vgg_stack((1,1,2,2,2),((3,64),(64, 128),(128,256),(256,512),(512,512)))
print(vgg_net)

Sequential(
  (0): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (1): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (2): Sequential(
    (0): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (3): Sequential(
    (0): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=

In [7]:
test_x = Variable(torch.zeros(1,3,256,256))
test_y = vgg_net(test_x)
print(test_y.shape)

torch.Size([1, 512, 8, 8])


In [0]:
class vgg(nn.Module):
    def __init__(self):
        super(vgg, self).__init__()
        self.feature = vgg_net
        self.fc = nn.Sequential(nn.Linear(512, 100),nn.ReLU(True), nn.Linear(100,10))
        
    def forward(self, x):
        x = self.feature(x)
        x = x.view(x.shape[0], -1)
        x = self.fc(x)
        return x

In [11]:
from Myutils import train
def data_tf(x):
    x = np.array(x, dtype='float32') / 255
    x = (x - 0.5) / 0.5
    x = x.transpose((2,0,1))
    x = torch.from_numpy(x)
    return x

train_set = CIFAR10('./data', train=True, transform=data_tf ,download=True)
train_data = torch.utils.data.DataLoader(train_set, batch_size=64, shuffle=True)

test_set = CIFAR10('./data', train=False, transform=data_tf)
test_data = torch.utils.data.DataLoader(test_set, batch_size=128, shuffle=False)

net = vgg().cuda()
optimizer = torch.optim.SGD(net.parameters(), lr=1e-1)
criterion = nn.CrossEntropyLoss()

Files already downloaded and verified


In [13]:
train(net, train_data, test_data, 20, optimizer, criterion)

/content/Myutils.py:70: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  im = Variable(im.cuda(), volatile=True)
/content/Myutils.py:71: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  label = Variable(label.cuda(), volatile=True)


Epoch 0. Train Loss: 2.303134, Train Acc: 0.098865, Valid Loss: 2.302682, Valid Acc: 0.100969, Time 00:00:50
Epoch 1. Train Loss: 2.302916, Train Acc: 0.100204, Valid Loss: 2.302939, Valid Acc: 0.098892, Time 00:00:53
Epoch 2. Train Loss: 2.302743, Train Acc: 0.099944, Valid Loss: 2.302182, Valid Acc: 0.098892, Time 00:00:54
Epoch 3. Train Loss: 2.156731, Train Acc: 0.172434, Valid Loss: 1.941185, Valid Acc: 0.239320, Time 00:00:53
Epoch 4. Train Loss: 1.765902, Train Acc: 0.319293, Valid Loss: 1.675352, Valid Acc: 0.351661, Time 00:00:54
Epoch 5. Train Loss: 1.528886, Train Acc: 0.426830, Valid Loss: 1.586826, Valid Acc: 0.401305, Time 00:00:53
Epoch 6. Train Loss: 1.301065, Train Acc: 0.522798, Valid Loss: 1.533042, Valid Acc: 0.450949, Time 00:00:53
Epoch 7. Train Loss: 1.085549, Train Acc: 0.611713, Valid Loss: 1.378252, Valid Acc: 0.543612, Time 00:00:54
Epoch 8. Train Loss: 0.905532, Train Acc: 0.682844, Valid Loss: 0.983675, Valid Acc: 0.667820, Time 00:00:54
Epoch 9. Train Loss